In [29]:
import copy
import json
import glob
import os
import random

import matplotlib.pyplot as plt
import nltk as nltk
import numpy as np
from collections import Counter
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
from random import shuffle
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn import svm

from catboost import CatBoostRegressor, Pool


In [30]:
train = pd.read_csv('train.csv', index_col=0).sample(frac=1)
y_train = train['correct']
X_train = train.drop(columns=['correct'])

test = pd.read_csv('test.csv', index_col=0).sample(frac=1)
y_test = test['correct']
X_test = test.drop(columns=['correct'])

test_r = pd.read_csv('test_raw.csv', index_col=0).sample(frac=1)
y_test_r = test_r['correct']
X_test_r = test_r.drop(columns=['correct'])


with open(os.path.join(os.getcwd(), 'randomNames.txt'), 'r') as f:
        randomNames = json.load(f)

len(randomNames)

/tmp/ipykernel_61431/1479892705.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv', index_col=0).sample(frac=1)
/tmp/ipykernel_61431/1479892705.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test.csv', index_col=0).sample(frac=1)


25

In [18]:
y_test_pred = []
for ind in X_test_r.index:
        name = str(X_test_r['name'][ind])
        first = name.rpartition('.')[2][0]
        val = 0
        for i, v in enumerate(randomNames[first]):
               if v == name:
                       val = i
        val = 1 - val / len(randomNames[first])
        y_test_pred.append(val)

In [19]:
y_test_pred

[0.16825095057034223,
 0.623006833712984,
 0.12274881516587677,
 0.5370729343348679,
 0.33220256845165985,
 0.4486692015209125,
 0.16185503685503688,
 0.5551839464882944,
 0.45512449334105387,
 0.3085308056872038,
 0.3424447174447175,
 0.623006833712984,
 0.30060422960725075,
 0.8459964932787843,
 0.489406779661017,
 0.23237218318391084,
 0.25658965180605275,
 0.8723404255319149,
 0.5969889982628835,
 0.4237160120845922,
 0.06687276277253495,
 0.1583054626532887,
 0.046971569839307836,
 0.7273413897280967,
 0.5370729343348679,
 0.39173405211141055,
 0.7085106382978723,
 0.3237872589129164,
 0.09692270414344561,
 0.8580645161290322,
 0.06409013811485342,
 0.051903677188415265,
 0.5370729343348679,
 0.7026055705300989,
 0.5370729343348679,
 0.6625592417061612,
 0.7121705174096974,
 0.16961473225102985,
 0.7121705174096974,
 0.17804154302670627,
 0.02779386218876667,
 0.2611454604620892,
 0.36733556298773695,
 0.5007552870090635,
 0.2779258541313303,
 0.39146919431279625,
 0.2511848341232

In [20]:
mean_squared_error(y_test_r.tolist(), y_test_pred)

0.2638804279567999

In [21]:
r2_score(y_test_r.tolist(), y_test_pred)

-0.6492526747299987

In [22]:
median_absolute_error(y_test_r.tolist(), y_test_pred)

0.4541203974284045

In [23]:
regr = svm.SVR()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)



ValueError: could not convert string to float: 'A'

In [ ]:
median_absolute_error(y_test.tolist(), y_pred)

In [31]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(X_train, y_train, cat_features=[0,1,2,3,4,5])
# Get predictions
preds = model.predict(X_test)

CatBoostError: Invalid type for cat_feature[non-default value idx=24,feature_idx=2]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
median_absolute_error(y_test.tolist(), preds)